In [ ]:
!pip install -q transformers datasets sentence-transformers hnswlib accelerate bitsandbytes peft


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer
import hnswlib
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType
import torch


In [ ]:
from google.colab import files
uploaded = files.upload()  # Upload your 'ARS_DATA.csv' here

df = pd.read_csv("ARS_DATA.csv")
print(f"Dataset loaded with {len(df)} rows")
df.head()



Saving ARS_DATA.csv to ARS_DATA.csv
Dataset loaded with 30000 rows


User ID  Age  Gender  Height (cm)  Weight (kg)   BMI Dietary Preferences  \
0     1001   18   Other          169         49.1  17.2               Halal   
1     1002   18   Other          189         66.4  18.6         Gluten-Free   
2     1003   53   Other          193        102.9  27.6      No Restriction   
3     1004   48   Other          191        118.4  32.5                Keto   
4     1005   38  Female          170        102.0  35.3         Gluten-Free   

  Fitness Goal     Exercise Level        Meal Name  Calories  Protein (g)  \
0     Fat Loss    Athlete (Daily)       Beef Steak       529           31   
1     Fat Loss  Light (1-2x/week)  Grilled Chicken       592           29   
2    Endurance    Athlete (Daily)      Veggie Wrap       323           38   
3     Fat Loss    Athlete (Daily)      Quinoa Bowl       746           46   
4  Maintenance    Athlete (Daily)  Grilled Chicken       455           34   

   Carbs (g)  Fats (g)  Meal Time  \
0         22         8  Breakfast   
1         28         7      Snack   
2         17         9  Breakfast   
3         28        21      Snack   
4         57        14      Snack   

                                     Description  
0        Iron-rich steak with roasted vegetables  
1    Lean protein meal for post-workout recovery  
2  Quick and healthy wrap for busy professionals  
3       Balanced vegan bowl for energy and fiber  
4    Lean protein meal for post-workout recovery

In [ ]:
def clean_and_encode_data(df):
    df.dropna(inplace=True)
    label_cols = ['Gender', 'Dietary Preferences', 'Fitness Goal', 'Exercise Level', 'Meal Name', 'Meal Time']
    encoders = {col: LabelEncoder() for col in label_cols}
    for col in label_cols:
        df[col + '_original'] = df[col]
        df[col] = encoders[col].fit_transform(df[col])
    scaler = StandardScaler()
    df[['Height (cm)', 'Weight (kg)', 'BMI']] = scaler.fit_transform(df[['Height (cm)', 'Weight (kg)', 'BMI']])
    return df, encoders, scaler

df, encoders, scaler = clean_and_encode_data(df)
print("Data cleaned and encoded")
df.head()


Data cleaned and encoded


User ID  Age  Gender  Height (cm)  Weight (kg)       BMI  \
0     1001   18       2    -0.401140    -1.678328 -1.245429   
1     1002   18       2     0.959834    -0.768178 -1.068513   
2     1003   53       2     1.232029     1.152080  0.068805   
3     1004   48       2     1.095931     1.967532  0.688012   
4     1005   38       0    -0.333092     1.104731  1.041844   

   Dietary Preferences  Fitness Goal  Exercise Level  Meal Name  ...  \
0                    1             1               0          0  ...   
1                    0             1               2          3  ...   
2                    4             0               0         10  ...   
3                    2             1               0          7  ...   
4                    0             2               0          3  ...   

   Carbs (g)  Fats (g)  Meal Time  \
0         22         8          0   
1         28         7          3   
2         17         9          0   
3         28        21          3   
4         57        14          3   

                                     Description  Gender_original  \
0        Iron-rich steak with roasted vegetables            Other   
1    Lean protein meal for post-workout recovery            Other   
2  Quick and healthy wrap for busy professionals            Other   
3       Balanced vegan bowl for energy and fiber            Other   
4    Lean protein meal for post-workout recovery           Female   

  Dietary Preferences_original Fitness Goal_original Exercise Level_original  \
0                        Halal              Fat Loss         Athlete (Daily)   
1                  Gluten-Free              Fat Loss       Light (1-2x/week)   
2               No Restriction             Endurance         Athlete (Daily)   
3                         Keto              Fat Loss         Athlete (Daily)   
4                  Gluten-Free           Maintenance         Athlete (Daily)   

  Meal Name_original Meal Time_original  
0         Beef Steak          Breakfast  
1    Grilled Chicken              Snack  
2        Veggie Wrap          Breakfast  
3        Quinoa Bowl              Snack  
4    Grilled Chicken              Snack  

[5 rows x 22 columns]

In [ ]:
def create_description(df):
    df['Full_Description'] = df.apply(lambda row:
        f"{row['Gender_original']}, {row['Age']} years old, {row['Height (cm)']:.2f} cm, {row['Weight (kg)']:.2f} kg, "
        f"{row['Dietary Preferences_original']} diet, {row['Fitness Goal_original']}, "
        f"{row['Exercise Level_original']} exercise, BMI {row['BMI']:.2f}",
        axis=1)
    return df

df = create_description(df)
print("Profile descriptions created")
df[['Full_Description']].head()


Profile descriptions created


Full_Description
0  Other, 18 years old, -0.40 cm, -1.68 kg, Halal...
1  Other, 18 years old, 0.96 cm, -0.77 kg, Gluten...
2  Other, 53 years old, 1.23 cm, 1.15 kg, No Rest...
3  Other, 48 years old, 1.10 cm, 1.97 kg, Keto di...
4  Female, 38 years old, -0.33 cm, 1.10 kg, Glute...

In [ ]:
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = embedder.encode(df['Full_Description'].tolist(), convert_to_numpy=True)

def build_hnsw_index(embeddings, ef=100, M=16):
    dim = embeddings.shape[1]
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=len(embeddings), ef_construction=ef, M=M)
    index.add_items(embeddings, ids=list(range(len(embeddings))))
    index.set_ef(ef)
    return index

index = build_hnsw_index(embeddings)
print("HNSW index built")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

HNSW index built


In [ ]:
def augment_with_similar_profiles(df, embeddings, index, top_k=3):
    similar_info = []
    for i, emb in enumerate(embeddings):
        ids, _ = index.knn_query([emb], k=top_k+1)
        neighbors = [df.iloc[idx]['Full_Description'] for idx in ids[0] if idx != i][:top_k]
        combined = " | ".join(neighbors)
        similar_info.append(combined)
    df['Similar_Profiles_Context'] = similar_info
    return df

df_augmented = augment_with_similar_profiles(df.copy(), embeddings, index)
print("Data augmented with similar profiles context")
df_augmented[['Full_Description', 'Similar_Profiles_Context']].head()


Data augmented with similar profiles context


Full_Description  \
0  Other, 18 years old, -0.40 cm, -1.68 kg, Halal...   
1  Other, 18 years old, 0.96 cm, -0.77 kg, Gluten...   
2  Other, 53 years old, 1.23 cm, 1.15 kg, No Rest...   
3  Other, 48 years old, 1.10 cm, 1.97 kg, Keto di...   
4  Female, 38 years old, -0.33 cm, 1.10 kg, Glute...   

                            Similar_Profiles_Context  
0  Other, 23 years old, -0.20 cm, 1.64 kg, Halal ...  
1  Other, 52 years old, -0.47 cm, 0.98 kg, Gluten...  
2  Other, 38 years old, 1.23 cm, 1.47 kg, No Rest...  
3  Other, 58 years old, 1.10 cm, 1.94 kg, Keto di...  
4  Female, 30 years old, 0.42 cm, 1.24 kg, Gluten...

In [ ]:
def prepare_finetuning_data_basic(df):
    df['input'] = df['Full_Description'].apply(lambda x: f"User info: {x}\n=> Recommended meal and workout:")
    df['output'] = df['Meal Name_original'].astype(str)
    df['text'] = df['input'] + " " + df['output']
    return Dataset.from_pandas(df[['text']])

def prepare_finetuning_data_augmented(df):
    df['input'] = df.apply(lambda row:
        f"User info: {row['Full_Description']}\nSimilar users: {row['Similar_Profiles_Context']}\n=> Recommended meal and workout:",
        axis=1)
    df['output'] = df['Meal Name_original'].astype(str)
    df['text'] = df['input'] + " " + df['output']
    return Dataset.from_pandas(df[['text']])

ds_basic = prepare_finetuning_data_basic(df)
ds_augmented = prepare_finetuning_data_augmented(df_augmented)

print(f"Basic dataset size: {len(ds_basic)}")
print(f"Augmented dataset size: {len(ds_augmented)}")


Basic dataset size: 30000
Augmented dataset size: 30000


In [ ]:
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType

model_name = "facebook/opt-350m"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# 8-bit quantization config
quant_config = BitsAndBytesConfig(load_in_8bit=True)

# Load model with 8-bit quantization and device map auto (works on GPU if available)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

# Apply LoRA on the base model
model = get_peft_model(base_model, lora_config)

# Set pad token to eos_token if pad_token is not defined
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Data collator for causal language modeling (no MLM)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

print("Model and tokenizer loaded with LoRA and 8-bit quantization")


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Model and tokenizer loaded with LoRA and 8-bit quantization


In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=128)

tokenized_ds_basic = ds_basic.map(tokenize_function, batched=True)
tokenized_ds_augmented = ds_augmented.map(tokenize_function, batched=True)

# Optionally reduce dataset size for faster runs
small_ds_basic = tokenized_ds_basic.select(range(min(5000, len(tokenized_ds_basic))))
small_ds_augmented = tokenized_ds_augmented.select(range(min(5000, len(tokenized_ds_augmented))))

print("Datasets tokenized")


Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

Datasets tokenized


In [ ]:
training_args_step1 = TrainingArguments(
    output_dir="./falcon1b_finetuned_step1",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir='./logs_step1',
    report_to="none",
    fp16=True,
    learning_rate=3e-4,
)

trainer_step1 = Trainer(
    model=model,
    args=training_args_step1,
    train_dataset=small_ds_basic,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Starting Step 1 fine-tuning...")
trainer_step1.train()
trainer_step1.save_model("./falcon1b_finetuned_step1")
tokenizer.save_pretrained("./falcon1b_finetuned_step1")
print("Step 1 fine-tuning complete and saved.")


<ipython-input-13-347ebea8d6e3>:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_step1 = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting Step 1 fine-tuning...


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step 1 fine-tuning complete and saved.


In [ ]:
# Reload model from Step 1 checkpoint
model = AutoModelForCausalLM.from_pretrained(
    "./falcon1b_finetuned_step1",
    quantization_config=quant_config,
    device_map="auto"
)
model = get_peft_model(model, lora_config)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

training_args_step2 = TrainingArguments(
    output_dir="./falcon1b_finetuned_step2",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir='./logs_step2',
    report_to="none",
    fp16=True,
    learning_rate=3e-4,
)

trainer_step2 = Trainer(
    model=model,
    args=training_args_step2,
    train_dataset=small_ds_augmented,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

print("Starting Step 2 fine-tuning...")
trainer_step2.train()
trainer_step2.save_model("./falcon1b_finetuned_step2")
tokenizer.save_pretrained("./falcon1b_finetuned_step2")
print("Step 2 fine-tuning complete and saved.")


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
<ipython-input-14-8b78a446e187>:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer_step2 = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Starting Step 2 fine-tuning...


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step 2 fine-tuning complete and saved.


In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    "./falcon1b_finetuned_step2",
    device_map="auto",
    quantization_config=quant_config
)
model = get_peft_model(model, lora_config)
model.eval()

tokenizer = AutoTokenizer.from_pretrained("./falcon1b_finetuned_step2")

def recommend(input_text, max_new_tokens=50):
    prompt = f"User info: {input_text}\n=> Recommended meal and workout:"
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=True, top_p=0.9, temperature=0.7)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return result.split("=> Recommended meal and workout:")[-1].strip()

print("Recommendation function ready")


Recommendation function ready


/usr/local/lib/python3.11/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/peft/tuners/tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
test_user_profile = "Male, 30 years old, 185 cm, 85 kg, vegetarian diet, muscle gain, moderate exercise, BMI 24.5"
print("Recommendation:", recommend(test_user_profile))


Recommendation: 1 cup of organic milk
=> Recommended diet: 1-2 servings of vegetables daily
=> Recommended exercise: Walking, running, cycling, swimming, or swimming in the water
=> Recommended exercise: Pilates, Pilates training, or yoga


In [ ]:
# -----------------------------------------
# INSTALL REQUIRED LIBRARIES (Google Colab)
# -----------------------------------------
!pip install -q transformers datasets sentence-transformers hnswlib accelerate bitsandbytes peft

# -----------------------------------------
# IMPORT LIBRARIES
# -----------------------------------------
from google.colab import files, drive
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sentence_transformers import SentenceTransformer
import hnswlib
import numpy as np
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling,
    pipeline,
    BitsAndBytesConfig,
)
from peft import get_peft_model, LoraConfig, TaskType
import torch

# -----------------------------------------
# MOUNT GOOGLE DRIVE
# -----------------------------------------
drive.mount('/content/drive')

# -----------------------------------------
# STEP 1: UPLOAD AND LOAD DATASET
# -----------------------------------------
uploaded = files.upload()  # Upload ARS_DATA.csv
df = pd.read_csv("ARS_DATA.csv")

# -----------------------------------------
# STEP 2: CLEANING AND ENCODING
# -----------------------------------------
def clean_and_encode_data(df):
    df.dropna(inplace=True)

    label_cols = ['Gender', 'Dietary Preferences', 'Fitness Goal', 'Exercise Level', 'Meal Name', 'Meal Time']
    encoders = {col: LabelEncoder() for col in label_cols}
    for col in label_cols:
        df[col + '_original'] = df[col]  # Store original values for later
        df[col] = encoders[col].fit_transform(df[col])

    scaler = StandardScaler()
    df[['Height (cm)', 'Weight (kg)', 'BMI']] = scaler.fit_transform(df[['Height (cm)', 'Weight (kg)', 'BMI']])

    return df, encoders, scaler

df, encoders, scaler = clean_and_encode_data(df)

# -----------------------------------------
# STEP 3: CREATE HUMAN-READABLE DESCRIPTIONS (USE ORIGINAL VALUES)
# -----------------------------------------
def create_description(df):
    df['Full_Description'] = df.apply(lambda row:
        f"{row['Gender_original']}, {row['Age']} years old, {row['Height (cm)']:.2f} cm, {row['Weight (kg)']:.2f} kg, "
        f"{row['Dietary Preferences_original']} diet, {row['Fitness Goal_original']}, "
        f"{row['Exercise Level_original']} exercise, BMI {row['BMI']:.2f}",
        axis=1)
    return df

df = create_description(df)

# -----------------------------------------
# STEP 4: GENERATE EMBEDDINGS
# -----------------------------------------
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')
embeddings = embedder.encode(df['Full_Description'].tolist(), convert_to_numpy=True)

# -----------------------------------------
# STEP 5: BUILD HNSW INDEX
# -----------------------------------------
def build_hnsw_index(embeddings, ef=100, M=16):
    dim = embeddings.shape[1]
    index = hnswlib.Index(space='cosine', dim=dim)
    index.init_index(max_elements=len(embeddings), ef_construction=ef, M=M)
    index.add_items(embeddings, ids=list(range(len(embeddings))))
    index.set_ef(ef)
    return index

index = build_hnsw_index(embeddings)

# -----------------------------------------
# STEP 6: PREPARE FOR GPT FINE-TUNING
# -----------------------------------------
def prepare_finetuning_data(df):
    df['input'] = df['Full_Description'].apply(lambda x: f"{x} => Recommended meal and workout:")
    df['output'] = df['Meal Name_original'].astype(str)
    df['text'] = df['input'] + " " + df['output']
    return Dataset.from_pandas(df[['text']])

ds = prepare_finetuning_data(df)

# -----------------------------------------
# STEP 7: LOAD MODEL + APPLY LoRA ADAPTERS (Required for 8-bit fine-tuning)
# -----------------------------------------
model_name = "facebook/opt-350m"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# Load 8-bit quantized base model
quant_config = BitsAndBytesConfig(load_in_8bit=True)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto"
)

# Add LoRA adapters
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # Adjust if needed
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)

# Ensure pad token is set
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# -----------------------------------------
# STEP 8: TOKENIZE DATA
# -----------------------------------------
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=64)

tokenized_dataset = ds.map(tokenize_function, batched=True)
small_ds = tokenized_dataset.select(range(min(5000, len(tokenized_dataset))))

# -----------------------------------------
# STEP 9: TRAINING ARGUMENTS & TRAINER
# -----------------------------------------
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/facebook/opt-350m",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    save_strategy="epoch",
    logging_dir='/content/drive/MyDrive/facebook_opt-350m_logs',
    report_to="none",
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# -----------------------------------------
# STEP 10: TRAIN & SAVE MODEL
# -----------------------------------------
trainer.train()

# Save PEFT-adapted model (only LoRA weights)
model.save_pretrained("/content/drive/MyDrive/facebook_opt-350m")
tokenizer.save_pretrained("/content/drive/MyDrive/facebook_opt-350mk")

# Save processed data, embeddings and index
df.to_csv("/content/drive/MyDrive/ARS_dfs.csv", index=False)
np.save("/content/drive/MyDrive/ARSs_embeddings.npy", embeddings)
index.save_index("/content/drive/MyDrive/ARSs_hnsw_index.bin")


Mounted at /content/drive


Saving ARS_DATA.csv to ARS_DATA.csv


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

Map:   0%|          | 0/30000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

<ipython-input-7-11ddd2a12d23>:157: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.11/dist-packages/bitsandbytes/autograd/_functions.py:315: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


In [ ]:
# Install required packages
!pip install -q transformers datasets sentence-transformers hnswlib accelerate bitsandbytes peft

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import libraries
import pandas as pd
import numpy as np
import hnswlib
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from peft import PeftModel

# Load dataset and embeddings
df = pd.read_csv("/content/drive/MyDrive/ARS_dfs.csv")
embeddings = np.load("/content/drive/MyDrive/ARSs_embeddings.npy")

# Load HNSW index
dim = embeddings.shape[1]
index = hnswlib.Index(space='cosine', dim=dim)
index.load_index("/content/drive/MyDrive/ARSs_hnsw_index.bin")
index.set_ef(100)

# Load sentence transformer for embeddings
embedder = SentenceTransformer('paraphrase-MiniLM-L3-v2')

# Load base model and LoRA fine-tuned adapter
model_name = "/content/drive/MyDrive/facebook_opt-350m"

tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/facebook_opt-350mk")

base_model = AutoModelForCausalLM.from_pretrained(
    "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",

)
model = PeftModel.from_pretrained(base_model, model_name)
model.eval()

# Setup generation pipeline (remove device argument)
gen_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

# Define recommendation function
def recommend(query, embedder, index, df, gen_pipeline, top_k=5):
    query_vec = embedder.encode([query])
    ids, distances = index.knn_query(query_vec, k=top_k)
    similar_profiles = df.iloc[ids[0]]['Full_Description'].tolist()
    context = "\n".join(similar_profiles)
    prompt = f"Based on similar users:\n\n{query} => Recommended meal and workout:\n"
    response = gen_pipeline(prompt, max_new_tokens=50, do_sample=True)[0]['generated_text']
    return response

# Example usage
query = "Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23"
response = recommend(query, embedder, index, df, gen_pipeline)
print("\n--- Recommendation Output ---\n")
print(response)

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 50.5 MB/s e

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.83k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/peft/peft_model.py:569: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.1.self_attn.v_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.q_proj.lora_B.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.lora_A.default.weight', 'base_model.model.model.layers.2.self_attn.v_proj.l


--- Recommendation Output ---

Based on similar users:
Male, 35 years old, 1.23 cm, 0.11 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI -0.60
Male, 64 years old, -0.40 cm, 1.26 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 1.23
Male, 23 years old, 0.89 cm, -1.37 kg, Vegetarian diet, Fat Loss, High (5x/week) exercise, BMI -1.45
Male, 46 years old, 0.35 cm, -1.39 kg, Vegetarian diet, Fat Loss, Sedentary exercise, BMI -1.30
Male, 25 years old, -1.15 cm, -0.26 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 0.42

Male, 30 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23 => Recommended meal and workout:arian diet, look Loss, publiclyentary exercise, BMI MarketJapan stone 30Japan stone, hopefully e 23 15 care Tuesday O, though t Market quarter've O community Market reported O put quarter G O here reported O put quarter G O here reported O


In [ ]:
query = "Male, 25 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23"
response = recommend(query, embedder, index, df, gen_pipeline)
print("\n--- Recommendation Output ---\n")
print(response)


--- Recommendation Output ---

Based on similar users:
Male, 64 years old, -0.40 cm, 1.26 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 1.23
Male, 25 years old, -1.15 cm, -0.26 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 0.42
Male, 35 years old, 1.23 cm, 0.11 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI -0.60
Male, 23 years old, 0.89 cm, -1.37 kg, Vegetarian diet, Fat Loss, High (5x/week) exercise, BMI -1.45
Male, 34 years old, -1.35 cm, 1.37 kg, Vegetarian diet, Fat Loss, Moderate (3x/week) exercise, BMI 2.20

Male, 25 years old, 160 cm, 80 kg, vegetarian diet, weight loss, moderate exercise, BMI 23 => Recommended meal and workout:arian diet, has Loss, its people (3x/week) exercise, BMI 2.20, disparate workout:arian diet, has Loss, its people (3x/week) exercise, BMI.�, gap, somehow, Senate,
